# Experiment README

## Table of Contents

* [Overview of Experiment Architecture](#Overview-of-Experiment-Architecture)
* [Experiment Workflow](#Experiment-Workflow)
    * [Modifying State Variables](#Modifying-State-Variables)
    * [Modifying System Parameters](#Modifying-System-Parameters)
    * [Executing Experiments](#Executing-Experiments)
    * [Post-processing and Analysing Results](#Post-processing-and-Analysing-Results)
    * [Visualizing Results](#Visualizing-Results)
* [Creating New, Customized Experiment Notebooks](#Creating-New,-Customized-Experiment-Notebooks)
    * Step 1: Select an experiment template
    * Step 2: Create a new notebook
    * Step 3: Customize the experiment
    * Step 4: Execute the experiment
* [Advanced Experiment-configuration & Simulation Techniques](#Advanced-Experiment-configuration-&-Simulation-Techniques)
    * [Setting Simulation Timesteps and Unit of Time `dt`](#Setting-Simulation-Timesteps-and-Unit-of-Time-dt)
    * [Changing the Ethereum Network Upgrade Stage](#Changing-the-Ethereum-Network-Upgrade-Stage)
    * [Performing Large-scale Experiments](#Performing-Large-scale-Experiments)

# Overview of Experiment Architecture

The experiment architecture is composed of the following four elements – the **model**, **default experiment**, **experiment templates**, and **experiment notebooks**:

1. The **model** is initialized with a default Initial State and set of System Parameters defined in the `model` module.
2. The **default experiment** – in the `experiments.default_experiment` module – is an experiment composed of a single simulation that uses the default cadCAD **model** Initial State and System Parameters. Additional default simulation execution settings such as the number of timesteps and runs are also set in the **default experiment**.
3. The **experiment templates** – in the `experiments.templates` module – contain pre-configured analyses based on the **default experiment**. Examples include `experiments.templates.time_domain_analysis` (simulation in the time-domain over a period of 5 years) and `experiments.templates.eth_price_sweep_analysis` (simulation in the phase-space sweeping over discrete ETH Price values).
4. The **experiment notebooks** perform various scenario analyses by importing existing **experiment templates**, optionally modifying the Initial State and System Parameters within the notebook, and then executing them.

# Experiment Workflow

If you just want to run (execute) existing experiment notebooks, simply open the respective notebook and execute all cells.

Depending on the chosen template and planned analysis, the required imports might differ slightly from the below standard dependencies:

In [1]:
# Import the setup module:
# * sets up the Python path
# * runs shared notebook-configuration methods, such as loading IPython modules
import setup

# External dependencies
import copy
import logging
import numpy as np
import pandas as pd
import plotly.express as px
from pprint import pprint

# Project dependencies
from experiments.run import run
from experiments.utils import display_code

time: 454 ms (started: 2021-09-13 11:56:09 +02:00)


We can then import the default experiment, and create a copy of the simulation object – we create a new copy for each analysis we'd like to perform:

In [2]:
import experiments.default_experiment as default_experiment
import experiments.templates.time_domain_analysis as time_domain_analysis

simulation_analysis_1 = copy.deepcopy(default_experiment.experiment.simulations[0])
simulation_analysis_2 = copy.deepcopy(time_domain_analysis.experiment.simulations[0])

time: 16.5 ms (started: 2021-09-13 11:56:09 +02:00)


We can use the `display_code` method to see the configuration of the default experiment before making changes:

In [3]:
display_code(default_experiment)

"""
The default experiment with default model Initial State, System Parameters, and Simulation Configuration.

The defaults are defined in their respective modules:
* Initial State in `model/state_variables.py`
* System Parameters in `model/system_parameters.py`
* Simulation Configuration in `experiments/simulation_configuration.py`
"""

from radcad import Simulation, Experiment, Backend

from model import model
from experiments.simulation_configuration import TIMESTEPS, MONTE_CARLO_RUNS


# Create Model Simulation
simulation = Simulation(
    model=model,
    timesteps=TIMESTEPS,
    runs=MONTE_CARLO_RUNS
)
# Create Experiment of single Simulation
experiment = Experiment([simulation])
# Configure Simulation & Experiment engine
simulation.engine = experiment.engine
experiment.engine.backend = Backend.SINGLE_PROCESS
experiment.engine.deepcopy = False
experiment.engine.drop_substeps = True

time: 86.9 ms (started: 2021-09-13 11:56:09 +02:00)


## Modifying State Variables

To view what the Initial State (radCAD model-configuration setting `initial_state`) of the State Variables are, and to what value they have been set, we can inspect the dictionary as follows:

In [4]:
pprint(simulation_analysis_1.model.initial_state)

{'avg_price': 5,
 'clients': 1,
 'hosts': 1,
 'indicated_network_demand': 10,
 'network_allocation': 0,
 'network_capacity': 1000,
 'network_penetration': 0,
 'potential_users': 10000,
 'timestamp': 0}
time: 15 ms (started: 2021-09-13 11:56:09 +02:00)


To modify the value of **State Variables** for a specific analysis, you need to select the relevant simulation and update the chosen model Initial State. For example, updating the `eth_supply` Initial State to `100e6` (100 million ETH):

In [5]:
simulation_analysis_1.model.initial_state.update({
    "hosts": 10, 
    "clients": 100
})

time: 21.9 ms (started: 2021-09-13 11:56:09 +02:00)


## Modifying System Parameters

To view what the System Parameters (radCAD model configuration setting `params`) are, and to what value they have been set, we can inspect the dictionary as follows:

In [6]:
pprint(simulation_analysis_1.model.params)

{'MIN_min_expected_fulfillment': [0.33],
 'avg_client_allocation': [10],
 'avg_host_line': [1000],
 'avg_reserve_capacity': [0.25],
 'client_competitor_price': [0.5],
 'client_registration_delay': [7],
 'host_line_cost': [0.5],
 'host_setup_delay': [60],
 'host_technical_difficulty': [0.25],
 'initial_population': [10000],
 'network_inefficiencies': [0.25],
 'onboarding_coefficient': [0.1],
 'price_change_delay': [14]}
time: 24.6 ms (started: 2021-09-13 11:56:09 +02:00)


To modify the value of **System Parameters** for a specific analysis, you need to select the relevant simulation, and update the chosen model System Parameter (which is a list of values). For example, updating the `BASE_REWARD_FACTOR` System Parameter to a sweep of two values, `64` and `32`:

In [12]:
simulation_analysis_1.model.params.update({
    "avg_client_allocation": [30],
})

time: 14.1 ms (started: 2021-09-13 11:56:30 +02:00)


## Executing Experiments

We can now execute our custom analysis and retrieve the post-processed Pandas DataFrame using the `run(...)` method:

In [13]:
df, exceptions = run(simulation_analysis_1)

2021-09-13 11:56:34,130 - root - INFO - Running experiment
2021-09-13 11:56:34,131 - root - INFO - Starting simulation 0 / run 0 / subset 0
2021-09-13 11:56:34,138 - root - INFO - Experiment complete in 0.006992816925048828 seconds
2021-09-13 11:56:34,139 - root - INFO - Post-processing results
2021-09-13 11:56:34,145 - root - INFO - Post-processing complete in 0.0069561004638671875 seconds
time: 30 ms (started: 2021-09-13 11:56:34 +02:00)


## Post-processing and Analysing Results

We can see that we had no exceptions for the single simulation we executed:

In [14]:
exceptions[0]['exception'] == None

True

time: 15.2 ms (started: 2021-09-13 11:56:36 +02:00)


We can simply display the Pandas DataFrame to inspect the results. This DataFrame already has some default post-processing applied (see [experiments/post_processing.py](../post_processing.py))

In [15]:
df

,timestamp,clients,hosts,potential_users,avg_price,network_capacity,indicated_network_demand,network_allocation,network_penetration,simulation,subset,run,substep,timestep
1,0,100.000000,18.616288,9881.383712,-0.352381,7500.000000,3.000000e+02,0.000000e+00,2.000000,0,0,1,1,1
2,0,337.759914,18.616288,9525.007510,0.023828,13962.215909,0.000000e+00,0.000000e+00,3.767952,0,0,1,1,2
3,0,462.498445,18.616288,9043.892777,-0.001702,13962.215909,2.126267e+05,8.011673e+03,3.908929,0,0,1,1,3
4,0,585.240236,18.616288,8440.036254,-0.002347,13962.215909,0.000000e+00,0.000000e+00,4.116875,0,0,1,1,4
5,0,697.711110,18.616288,7723.708856,0.000168,13962.215909,0.000000e+00,0.000000e+00,4.411424,0,0,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0,-75.562886,18.616288,586.550801,0.000674,13962.215909,-0.000000e+00,-0.000000e+00,70.302644,0,0,1,1,361
362,0,-66.382463,18.616288,634.316976,-0.000048,13962.215909,-1.680956e+06,-1.181757e+06,63.477154,0,0,1,1,362
363,0,-56.637480,18.616288,672.338168,0.000556,13962.215909,-0.000000e+00,-0.000000e+00,58.697114,0,0,1,1,363
364,0,-46.500767,18.616288,700.222647,-0.000040,13962.215909,-1.528868e+06,-8.974014e+05,55.377751,0,0,1,1,364


time: 30.4 ms (started: 2021-09-13 11:56:38 +02:00)


## Visualizing Results

Once we have the results post-processed and in a Pandas DataFrame, we can use Plotly for plotting our results:

In [18]:
# Plot the total validating rewards in ETH for each subset
px.line(df, x='timestep', y='network_allocation')

time: 78.2 ms (started: 2021-09-13 11:57:55 +02:00)


# Creating New, Customized Experiment Notebooks

If you want to create an entirely new analysis, you'll need to create a new experiment notebook, which entails the following steps:
* Step 1: Select an experiment template from the `experiments/templates/` directory to start from. If you'd like to create your own template, the [example_analysis.py](../templates/example_analysis.py) template gives an example of extending the default experiment to override default State Variables and System Parameters that you can copy.
* Step 2: Create a new notebook in the `experiments/notebooks/` directory, using the [template.ipynb](./template.ipynb) notebook as a guide, and import the experiment from the experiment template.
* Step 3: Customize the experiment for your specific analysis.
* Step 4: Execute your experiment, post-process and analyze the results, and create Plotly charts!

# Advanced Experiment-configuration & Simulation Techniques

## Setting Simulation Timesteps and Unit of Time `dt`

In [ ]:
from experiments.simulation_configuration import TIMESTEPS, DELTA_TIME, SIMULATION_TIME_MONTHS

We can configure the number of simulation timesteps `TIMESTEPS` from a simulation time in months `SIMULATION_TIME_MONTHS`, multiplied by the number of epochs in a month, and divided by the simulation unit of time `DELTA_TIME`:

In [ ]:
SIMULATION_TIME_MONTHS / 12  # Divide months by 12 to get number of years

`DELTA_TIME` is a variable that sets how many epochs are simulated for each timestep. Sometimes, if we don't need a finer granularity (1 epoch per timestep, for example), we can then set `DELTA_TIME` to a larger value for better performance. The default value is 1 day or `225` epochs. This means that all our time-based states will be for a period of 1 day (we call this "aggregation"), which is convenient.

In [ ]:
DELTA_TIME

`TIMESTEPS` is now simply the simulation time in months, multiplied by the number of epochs in a month, divided by `DELTA_TIME`:

```python
TIMESTEPS = constants.epochs_per_month * SIMULATION_TIME_MONTHS // DELTA_TIME
```

In [ ]:
TIMESTEPS

Finally, to set the simulation timesteps (note, you may have to update the environmental processes that depend on the number of timesteps, and override the relevant parameters):

In [ ]:
simulation_analysis_1.timesteps = TIMESTEPS

### Considerations When Performing Efficient Phase-space Simulations

In `simulation_analysis_3`, `timesteps` is decreased to `1`, but `dt` is increased to `TIMESTEPS * DELTA_TIME`, where `DELTA_TIME` is the full duration of the simulation. This produces the final result in a single processing cycle, producing the full phase-space with very low processing overhead. This is achieved by ignoring all time-series information between the beginning and end of the simulation.

There is a test function `test_dt(...)` in `tests/test_integration.py` that can be used to verify that no information is lost due to the approximations taken along the time axis for the specific State Variables that you are interested in, and that your custom code has not introduced mechanisms that might not work well with this kind of approximation. 

An example of a type of mechanism that would not work with this kind of approximation is a mechanism that implements some form of feedback loop.